In [1]:
from transformers import (
    AutoConfig,
    AutoModelForMultipleChoice,
)

/u/lucmon/lucmon/anaconda3/envs/mlopt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "roberta-base"

config = AutoConfig.from_pretrained(
        model_name,
        #cache_dir=model_args.cache_dir,
        revision="main",
        use_auth_token=None,
    )

model = AutoModelForMultipleChoice.from_pretrained(
        model_name,
        from_tf=bool(".ckpt" in model_name),
        config=config,
        #cache_dir=model_args.cache_dir,
        revision="main",
        use_auth_token=None,
    )

Some weights of RobertaForMultipleChoice were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join('..', 'arch'))

# Add to sys.path if not already present
if module_path not in sys.path:
    sys.path.append(module_path)
from lora import add_adapters_dataset

model , output_layer_name, Lora_Config = add_adapters_dataset(model_name, model, 16, 16, lora_freeze_a=False)

The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
/u/lucmon/lucmon/anaconda3/envs/mlopt/lib/python3.11/site-packages/torch/cuda/__init__.py:716: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [8]:
base_name = "base_model.model.roberta.encoder.layer.0.attention.self.value.base_layer.weight"
#base_name = "base_model.model.roberta.encoder.layer.0.attention.self.value.weight"
def base_to_lora_name(base_name):
    lora_A_name = base_name.replace("base_layer", "lora_A.default")
    lora_B_name = base_name.replace("base_layer", "lora_B.default")
    return lora_A_name, lora_B_name
lora_A_name, lora_B_name = base_to_lora_name(base_name)

#lora_A_name, lora_B_name = None, None
for name, param in model.named_parameters():
    if name == base_name:
        print("???")
        base_param = param.clone()
    if name == lora_A_name:
        print("A")
        lora_A_param = param
    if name == lora_B_name:
        print('B')
        lora_B_param = param
print(lora_A_param.shape, lora_B_param.shape)

???
A
B
torch.Size([16, 768]) torch.Size([768, 16])


In [7]:
lora_A_name

In [9]:
import torch
lora_B_param.data = torch.randn_like(lora_B_param)

scaling = 1
my_product = scaling * lora_B_param @ lora_A_param
merged_model = model.merge_and_unload()
merged_base_name = "roberta.encoder.layer.0.attention.self.value.weight"
for name, param in merged_model.named_parameters():
    if name == merged_base_name:
        real_update = param - base_param
        break

In [11]:
print(my_product)

tensor([[ 0.0560, -0.0255,  0.1228,  ...,  0.0728, -0.1704,  0.0258],
        [ 0.0490, -0.1293, -0.0183,  ..., -0.0340, -0.0298,  0.1013],
        [ 0.0850,  0.0555, -0.0552,  ...,  0.1703, -0.1314, -0.1007],
        ...,
        [-0.1876,  0.0186,  0.0640,  ..., -0.0252,  0.0345, -0.1034],
        [ 0.0117, -0.0402,  0.0104,  ..., -0.0239,  0.0612,  0.0206],
        [ 0.0506,  0.0426,  0.0024,  ...,  0.0611, -0.1517, -0.1562]],
       grad_fn=<MmBackward0>)


In [10]:
print(real_update)

tensor([[ 0.0560, -0.0255,  0.1228,  ...,  0.0728, -0.1704,  0.0258],
        [ 0.0490, -0.1293, -0.0183,  ..., -0.0340, -0.0298,  0.1013],
        [ 0.0850,  0.0555, -0.0552,  ...,  0.1703, -0.1314, -0.1007],
        ...,
        [-0.1876,  0.0186,  0.0640,  ..., -0.0252,  0.0345, -0.1034],
        [ 0.0117, -0.0402,  0.0104,  ..., -0.0239,  0.0612,  0.0206],
        [ 0.0506,  0.0426,  0.0024,  ...,  0.0611, -0.1517, -0.1562]])


In [5]:
for name, param in model.named_parameters():
    print(name)

base_model.model.roberta.embeddings.word_embeddings.weight
base_model.model.roberta.embeddings.position_embeddings.weight
base_model.model.roberta.embeddings.token_type_embeddings.weight
base_model.model.roberta.embeddings.LayerNorm.weight
base_model.model.roberta.embeddings.LayerNorm.bias
base_model.model.roberta.encoder.layer.0.attention.self.query.base_layer.weight
base_model.model.roberta.encoder.layer.0.attention.self.query.base_layer.bias
base_model.model.roberta.encoder.layer.0.attention.self.query.lora_A.default.weight
base_model.model.roberta.encoder.layer.0.attention.self.query.lora_B.default.weight
base_model.model.roberta.encoder.layer.0.attention.self.key.weight
base_model.model.roberta.encoder.layer.0.attention.self.key.bias
base_model.model.roberta.encoder.layer.0.attention.self.value.base_layer.weight
base_model.model.roberta.encoder.layer.0.attention.self.value.base_layer.bias
base_model.model.roberta.encoder.layer.0.attention.self.value.lora_A.default.weight
base_mode

# ViT

In [12]:
model_name == "google/vit-base-patch16-224-in21k"        
from transformers import AutoModelForImageClassification
model = AutoModelForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k', num_labels=10)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
import os
import sys
module_path = os.path.abspath(os.path.join('..', 'arch'))

# Add to sys.path if not already present
if module_path not in sys.path:
    sys.path.append(module_path)
from lora import add_adapters_dataset

model , output_layer_name, Lora_Config = add_adapters_dataset(model_name, model, 16, 16, lora_freeze_a=False)

In [14]:
for name, param in model.named_parameters():
    print(name)

base_model.model.vit.embeddings.cls_token
base_model.model.vit.embeddings.position_embeddings
base_model.model.vit.embeddings.patch_embeddings.projection.weight
base_model.model.vit.embeddings.patch_embeddings.projection.bias
base_model.model.vit.encoder.layer.0.attention.attention.query.base_layer.weight
base_model.model.vit.encoder.layer.0.attention.attention.query.base_layer.bias
base_model.model.vit.encoder.layer.0.attention.attention.query.lora_A.default.weight
base_model.model.vit.encoder.layer.0.attention.attention.query.lora_B.default.weight
base_model.model.vit.encoder.layer.0.attention.attention.key.weight
base_model.model.vit.encoder.layer.0.attention.attention.key.bias
base_model.model.vit.encoder.layer.0.attention.attention.value.base_layer.weight
base_model.model.vit.encoder.layer.0.attention.attention.value.base_layer.bias
base_model.model.vit.encoder.layer.0.attention.attention.value.lora_A.default.weight
base_model.model.vit.encoder.layer.0.attention.attention.value.lo

In [15]:
base_name = "base_model.model.vit.encoder.layer.0.attention.attention.value.base_layer.weight"
#base_name = "base_model.model.roberta.encoder.layer.0.attention.self.value.weight"
def base_to_lora_name(base_name):
    lora_A_name = base_name.replace("base_layer", "lora_A.default")
    lora_B_name = base_name.replace("base_layer", "lora_B.default")
    return lora_A_name, lora_B_name
lora_A_name, lora_B_name = base_to_lora_name(base_name)

#lora_A_name, lora_B_name = None, None
for name, param in model.named_parameters():
    if name == base_name:
        print("???")
        base_param = param.clone()
    if name == lora_A_name:
        print("A")
        lora_A_param = param
    if name == lora_B_name:
        print('B')
        lora_B_param = param
print(lora_A_param.shape, lora_B_param.shape)

???
A
B
torch.Size([16, 768]) torch.Size([768, 16])


In [16]:
import torch
lora_B_param.data = torch.randn_like(lora_B_param)

scaling = 1
my_product = scaling * lora_B_param @ lora_A_param
merged_model = model.merge_and_unload()

In [17]:
for name, param in merged_model.named_parameters():
    print(name)

vit.embeddings.cls_token
vit.embeddings.position_embeddings
vit.embeddings.patch_embeddings.projection.weight
vit.embeddings.patch_embeddings.projection.bias
vit.encoder.layer.0.attention.attention.query.weight
vit.encoder.layer.0.attention.attention.query.bias
vit.encoder.layer.0.attention.attention.key.weight
vit.encoder.layer.0.attention.attention.key.bias
vit.encoder.layer.0.attention.attention.value.weight
vit.encoder.layer.0.attention.attention.value.bias
vit.encoder.layer.0.attention.output.dense.weight
vit.encoder.layer.0.attention.output.dense.bias
vit.encoder.layer.0.intermediate.dense.weight
vit.encoder.layer.0.intermediate.dense.bias
vit.encoder.layer.0.output.dense.weight
vit.encoder.layer.0.output.dense.bias
vit.encoder.layer.0.layernorm_before.weight
vit.encoder.layer.0.layernorm_before.bias
vit.encoder.layer.0.layernorm_after.weight
vit.encoder.layer.0.layernorm_after.bias
vit.encoder.layer.1.attention.attention.query.weight
vit.encoder.layer.1.attention.attention.query

In [18]:
merged_base_name = "vit.encoder.layer.0.attention.attention.value.weight"
for name, param in merged_model.named_parameters():
    if name == merged_base_name:
        real_update = param - base_param
        break

In [19]:
print(real_update)

tensor([[ 0.0321,  0.0991, -0.1132,  ..., -0.1183,  0.1426, -0.1396],
        [ 0.0086, -0.0049,  0.0900,  ...,  0.1232, -0.0167, -0.0734],
        [-0.0950, -0.0079,  0.0176,  ...,  0.0036, -0.0152,  0.1182],
        ...,
        [ 0.0895, -0.1026,  0.0633,  ...,  0.0311, -0.0354,  0.0450],
        [ 0.1237, -0.0355, -0.0320,  ..., -0.0396,  0.0303,  0.0164],
        [ 0.1022, -0.1361,  0.0503,  ...,  0.1041, -0.0256,  0.0526]])


In [20]:
print(my_product)

tensor([[ 0.0321,  0.0991, -0.1132,  ..., -0.1183,  0.1426, -0.1396],
        [ 0.0086, -0.0049,  0.0900,  ...,  0.1232, -0.0167, -0.0734],
        [-0.0950, -0.0079,  0.0176,  ...,  0.0036, -0.0152,  0.1182],
        ...,
        [ 0.0895, -0.1026,  0.0633,  ...,  0.0311, -0.0354,  0.0450],
        [ 0.1237, -0.0355, -0.0320,  ..., -0.0396,  0.0303,  0.0164],
        [ 0.1022, -0.1361,  0.0503,  ...,  0.1041, -0.0256,  0.0526]],
       grad_fn=<MmBackward0>)


#GPT2

In [21]:
from transformers import AutoModelForSequenceClassification
model_name = "gpt2"
model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=20,pad_token_id=50256)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
import os
import sys
module_path = os.path.abspath(os.path.join('..', 'arch'))

# Add to sys.path if not already present
if module_path not in sys.path:
    sys.path.append(module_path)
from lora import add_adapters_dataset

model , output_layer_name, Lora_Config = add_adapters_dataset(model_name, model, 16, 16, lora_freeze_a=False)

/u/lucmon/lucmon/anaconda3/envs/mlopt/lib/python3.11/site-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [26]:
for name, param in model.named_parameters():
    print(name, param.shape)

base_model.model.transformer.wte.weight torch.Size([50257, 768])
base_model.model.transformer.wpe.weight torch.Size([1024, 768])
base_model.model.transformer.h.0.ln_1.weight torch.Size([768])
base_model.model.transformer.h.0.ln_1.bias torch.Size([768])
base_model.model.transformer.h.0.attn.c_attn.base_layer.weight torch.Size([768, 2304])
base_model.model.transformer.h.0.attn.c_attn.base_layer.bias torch.Size([2304])
base_model.model.transformer.h.0.attn.c_attn.lora_A.default.weight torch.Size([16, 768])
base_model.model.transformer.h.0.attn.c_attn.lora_B.default.weight torch.Size([2304, 16])
base_model.model.transformer.h.0.attn.c_proj.base_layer.weight torch.Size([768, 768])
base_model.model.transformer.h.0.attn.c_proj.base_layer.bias torch.Size([768])
base_model.model.transformer.h.0.attn.c_proj.lora_A.default.weight torch.Size([16, 768])
base_model.model.transformer.h.0.attn.c_proj.lora_B.default.weight torch.Size([768, 16])
base_model.model.transformer.h.0.ln_2.weight torch.Size([7

In [28]:
base_name = "base_model.model.transformer.h.0.attn.c_attn.base_layer.weight"
#base_name = "base_model.model.roberta.encoder.layer.0.attention.self.value.weight"
def base_to_lora_name(base_name):
    lora_A_name = base_name.replace("base_layer", "lora_A.default")
    lora_B_name = base_name.replace("base_layer", "lora_B.default")
    return lora_A_name, lora_B_name
lora_A_name, lora_B_name = base_to_lora_name(base_name)

#lora_A_name, lora_B_name = None, None
for name, param in model.named_parameters():
    if name == base_name:
        print("???")
        base_param = param.clone()
    if name == lora_A_name:
        print("A")
        lora_A_param = param
    if name == lora_B_name:
        print('B')
        lora_B_param = param
print(lora_A_param.shape, lora_B_param.shape)

???
A
B
torch.Size([16, 768]) torch.Size([2304, 16])


In [29]:
import torch
lora_B_param.data = torch.randn_like(lora_B_param)

scaling = 1
my_product = scaling * lora_B_param @ lora_A_param
merged_model = model.merge_and_unload()

In [30]:
for name, param in merged_model.named_parameters():
    print(name)

transformer.wte.weight
transformer.wpe.weight
transformer.h.0.ln_1.weight
transformer.h.0.ln_1.bias
transformer.h.0.attn.c_attn.weight
transformer.h.0.attn.c_attn.bias
transformer.h.0.attn.c_proj.weight
transformer.h.0.attn.c_proj.bias
transformer.h.0.ln_2.weight
transformer.h.0.ln_2.bias
transformer.h.0.mlp.c_fc.weight
transformer.h.0.mlp.c_fc.bias
transformer.h.0.mlp.c_proj.weight
transformer.h.0.mlp.c_proj.bias
transformer.h.1.ln_1.weight
transformer.h.1.ln_1.bias
transformer.h.1.attn.c_attn.weight
transformer.h.1.attn.c_attn.bias
transformer.h.1.attn.c_proj.weight
transformer.h.1.attn.c_proj.bias
transformer.h.1.ln_2.weight
transformer.h.1.ln_2.bias
transformer.h.1.mlp.c_fc.weight
transformer.h.1.mlp.c_fc.bias
transformer.h.1.mlp.c_proj.weight
transformer.h.1.mlp.c_proj.bias
transformer.h.2.ln_1.weight
transformer.h.2.ln_1.bias
transformer.h.2.attn.c_attn.weight
transformer.h.2.attn.c_attn.bias
transformer.h.2.attn.c_proj.weight
transformer.h.2.attn.c_proj.bias
transformer.h.2.ln_2

In [31]:
merged_base_name = "transformer.h.0.attn.c_attn.weight"
for name, param in merged_model.named_parameters():
    if name == merged_base_name:
        real_update = param - base_param
        break

In [34]:
print(real_update)
print(real_update.shape)

tensor([[-0.0546, -0.0664,  0.0471,  ...,  0.1405,  0.0787,  0.0821],
        [ 0.1177, -0.0407,  0.0580,  ...,  0.0716, -0.0565, -0.0031],
        [-0.0689, -0.0574, -0.0427,  ...,  0.0564, -0.0220,  0.0140],
        ...,
        [ 0.1406,  0.1481,  0.0080,  ...,  0.0284,  0.0265,  0.0121],
        [ 0.0258,  0.0373, -0.0037,  ...,  0.0410,  0.0567, -0.0127],
        [-0.1159, -0.0073,  0.0870,  ...,  0.0939,  0.0538,  0.0147]])
torch.Size([768, 2304])


In [35]:
print(my_product)
print(my_product.shape)

tensor([[-0.0546,  0.1177, -0.0689,  ...,  0.1406,  0.0258, -0.1159],
        [-0.0664, -0.0407, -0.0574,  ...,  0.1481,  0.0373, -0.0073],
        [ 0.0471,  0.0580, -0.0427,  ...,  0.0080, -0.0037,  0.0870],
        ...,
        [ 0.1405,  0.0716,  0.0564,  ...,  0.0284,  0.0410,  0.0939],
        [ 0.0787, -0.0565, -0.0220,  ...,  0.0265,  0.0567,  0.0538],
        [ 0.0821, -0.0031,  0.0140,  ...,  0.0121, -0.0127,  0.0147]],
       grad_fn=<MmBackward0>)
torch.Size([2304, 768])
